# Sportsmith podcast summary generator

In [ ]:
## Install dependencies
#pip install youtube_transcript_api
#pip install google-api-python-client
#pip install pyairtable

In [ ]:
import os
from pyairtable import Table
at_api_key = ["YOUR API KEY HERE"]

In [ ]:
# Establish airtable instance
table = Table(at_api_key, 'YOUR BASE ID', 'YOUR TABLE ID')

In [ ]:
# Import dependencies

from transformers import pipeline
summarizer = pipeline("summarization",truncation=True)
import urllib.request
import json
import urllib
import re
from youtube_transcript_api import YouTubeTranscriptApi
from gensim.parsing.preprocessing import remove_stopwords
from collections import Counter
from apiclient.discovery import build

In [ ]:
api_key = ['YOUR YT API KEY HERE']
youtube = build('youtube', 'v3', developerKey = api_key)

# Function to get all channel videos IDs into a list
def get_channel_videos(channel_id):
    res = youtube.channels().list(id = channel_id,
                                    part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    videos = []
    next_page_token = None

    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                           part='snippet',
                                           pageToken = next_page_token,
                                           maxResults=50).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break

    return videos

In [ ]:
# Run function to get all video IDs from channel of interest
videos = get_channel_videos('UC2wKJNafA-Zxoz1AE-huebQ')

id_list = []

for video in videos:
    id_list.append(video['snippet']['resourceId']['videoId'])
    


In [ ]:
# Several videos did not have transcripts, this step removes them from the id_list

no_transcripts = ['ACqGgYp2yaI',
 'LtfoQPj9IXg',
 'O6JWPYagcvw',
 'WXiBe0ZQ19A',
 'GI1z5vuHN4A',
 '6fMfH1ZgwmU',
 'tGhjQiKZZtk',
 'foEsfubNED0',
 'DfUDsf_GMHQ',
 'tNoyBRQqvZ8',
 '-Zb_L01mhIM',
 'V7CbKYNx26E',
 'Cxqbzh3YBjE']

id_list = [x for x in id_list if x not in no_transcripts]

In [ ]:
published_list=[]

for video in videos:
    published_list.append(video['snippet']['publishedAt'])
    
id_date_dict = dict(zip(id_list,published_list))

In [ ]:
# In the event the function needs to be re-run (e.g. due to a timeout), this step removes video IDs that have already been created in airtable

table_dict = table.all()

finished_list = []

for i in range(0,len(table_dict)):
    #print(table_dict[i]['fields']['ID'])
    finished_list.append(table_dict[i]['fields']['ID'])

unfinished_list = [x for x in id_list if x not in finished_list]    
    
id_list = unfinished_list    

In [ ]:
# Create add_podcast function to extract necessary text data, punctuate, summarize and upload to airtable

def add_podcast(VideoID):
    VideoID = VideoID

    params = {"format": "json", "url": "https://www.youtube.com/watch?v=%s" % VideoID}
    url = "https://www.youtube.com/oembed"
    query_string = urllib.parse.urlencode(params)
    url = url + "?" + query_string
    url
    
    Published_date = id_date_dict[VideoID].split('T')[0]
    
    with urllib.request.urlopen(url) as response:
        response_text = response.read()
        data = json.loads(response_text.decode())
    
    raw_text =YouTubeTranscriptApi.get_transcript(VideoID)
    
    Title = data['title']
    Image = data['thumbnail_url']
    Full_link = data['author_url']
    Author = data['author_name']
    
    list_text = [text['text'] for text in raw_text]

    joined_string = ",".join(list_text)
    
    transformed_string=re.sub(","," ",joined_string)
    
    # remove additional words
    stop_words_lst = ['um', 'oh', 'yeah', 'yep','uh', 'know', 'like',"it's", 'going', 'think','kind','I']

    for w in stop_words_lst:
        pattern = r'\b'+w+r'\b'
        transformed_string = re.sub(pattern, '', transformed_string)
        
    filtered_sentence = remove_stopwords(transformed_string)
    
    # Tag video based on most common word
    counted_text = Counter(filtered_sentence.split())
    Tag=counted_text.most_common()[0][0]
    
    # Punctuate text

    import subprocess

    cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=transformed_string)
    
    Fulltext = cased
    
    split_tokens = cased.split(" ")
    docs = []
    for i in range(0, len(split_tokens), 850):
        selection = " ".join(split_tokens[i:(i+850)])
        docs.append(selection)
    
    summaries = summarizer(docs)
    summary = "\n\n".join([d["summary_text"] for d in summaries])
    
    table.create({'Title':Title,'ID':VideoID,'Author':Author,'Images':Image,'Summary':summary,'Fulltext':Fulltext,'Tag':Tag, 'Published_date': Published_date})
    

In [ ]:
# Run the add_podcast function on each video ID

for id in id_list:
    add_podcast(id)